In [1]:
!pip install langchain tiktoken transformers
!pip install sentence-transformers
!pip install -U langchain-community
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [2]:
!pip install pypdf

In [3]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 10.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.2 MB/s eta 0:00:00


In [4]:
from pinecone import Pinecone
pc = Pinecone(api_key="")

In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFDirectoryLoader('/kaggle/input/pinecorn',glob="./*.pdf")
data=loader.load()

In [8]:
data[500]

Document(metadata={'source': '/kaggle/input/pinecorn/Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'page': 500}, page_content='DailyDoseofDS.com\nThere’sredundancybecauseeverysettermethodwillhavealmostthesamelines\nofcode(theifstatementsforvalidation).\nAlso,ifyouthinkaboutit,thegettermethodsaresomewhatredundantand\nunnecessarytoo,astheyjustreturnanattribute.\nIfthatisclear,there’sonemoreissuewiththeaboveimplementation.\nRecallwhatImentionedearlier:“Ourclasswillhave3suchinstance-level\nattributes,andallmustbepositive?”\n500\n')

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(data)

In [10]:
len(text)

583

In [77]:
import os
from pinecone import Pinecone, ServerlessSpec

# Set API key and environment
api_key = ''  # Replace with your actual API key
api_env = 'us-east-1'  # Replace with your Pinecone environment (e.g., 'us-east-1')

# Initialize Pinecone
pc = Pinecone(api_key=api_key)

# Check if the index exists, create if not
index_name = "pinecorn"  # Replace with your index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Replace with the number of dimensions in your data
        metric='cosine',  # Replace with the desired metric (e.g., 'euclidean', 'dotproduct')
        spec=ServerlessSpec(
            cloud='aws',  # Replace with your desired cloud provider
            region=api_env
        )
    )

# Use your index
index = pc.Index(index_name)

In [18]:
text[555]

Document(metadata={'source': '/kaggle/input/pinecorn/Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'page': 511}, page_content='DailyDoseofDS.com\nCanwerestrictthisdynamicity?\nOfcoursewecan!\nDeﬁningaslottedclasshelpsusachievethis.Simplyput,itallowsustoﬁxthe\ninstance-levelattributesaclassobjectcaneverpossess.\nAslottedclassisdeclaredasfollows:\nDeﬁnea\x00_\x00\x00o\x00\x00_\x00attributewithallattributesanobjectmaypossess.\nDone!\n511')

In [19]:
texts = [doc.page_content for doc in text]
sources=[doc.metadata for doc in text]

In [21]:
texts[0],sources[0]

('DATASCIENCE\nFREE2024 EDITION\nFULL ARCHIVE\n Avi ChawlaDailyDoseofDS.comDaily Dose ofData Science',
 {'source': '/kaggle/input/pinecorn/Daily_Dose_Of_Data_Science_Full_Archive.pdf',
  'page': 0})

In [31]:
# data = [
#     {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
#     {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
#     {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
#     {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
#     {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
#     {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
# ]

embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[doc.page_content for doc in text[:96]],
    parameters={"input_type": "passage", "truncate": "END"}
)
print(embeddings[0])   //this model processes 96 requests at one time

{'vector_type': dense, 'values': [0.0232086181640625, -0.0099945068359375, ..., -0.0323486328125, 0.00988006591796875]}


In [61]:
text=text[:96]
len(text)

96

In [62]:
text[0]

Document(metadata={'source': '/kaggle/input/pinecorn/Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'page': 0}, page_content='DATASCIENCE\nFREE2024 EDITION\nFULL ARCHIVE\n Avi ChawlaDailyDoseofDS.comDaily Dose ofData Science')

In [64]:
embeddings

{'vector_type': dense, 'values': [0.0232086181640625, -0.0099945068359375, ..., -0.0323486328125, 0.00988006591796875]}

In [75]:
vectors = []
i = 0
for d, e in zip(text, embeddings):  # Iterate over text and corresponding embedding
    vectors.append({
        "id": f"doc{i}",          # Assign a unique ID for each vector
        "values": e["values"],    # Use the correct embedding values
        "metadata": {"text": str(d)}   # Add the text as metadata
    })
    i += 1


In [78]:
index.upsert(
    vectors=vectors,
)

{'upserted_count': 96}

In [79]:
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 96}},
 'total_vector_count': 96}


In [80]:
query = "Tell me about the tech company known as Apple."

embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

In [82]:
results = index.query(
    vector=embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'doc1',
              'metadata': {'text': "page_content='DailyDoseofDS.com\n"
                                   '\x00o\x00\x00o\x00a\x00\x00\x00h\x00\x00o\x00\x00\x00u\x00\x00f\x00h\x00\x00\x00o\x00\x00\n'
                                   '\x00n\x00\x00o\x00\x00\x00i\x00\x00?\n'
                                   'Readingtimeofthisbookisabout9-10hours.Butnotallchapterswillbeof\n'
                                   'relevancetoyou.This2-minuteassessmentwilltestyourcurrentexpertiseand\n'
                                   'recommendchaptersthatwillbemostusefultoyou.\n'
                                   'ScantheQRcodebeloworopenthislinktostarttheassessment.Itwillonlytake\n'
                                   '2minutestocomplete.\n'
                                   'https://bit.ly/DS-assessment\n'
                                   "1' metadata={'source': "
                                   "'/kaggle/input/pinecorn/Daily_Dose_Of_Data_Science_Full_Archive.pdf', "

In [83]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [84]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.1-70b-versatile"
)